**Analysis owner**: Felipe Matheus <br>
**Date**: 12/10/2021 <br>
**Related Links**:

The objective of this notebook is to download porcesssed data from eda notebook and do a simple classification (identification) model.

# Index

1. Setup
2. Loading data
3. Launching H2O AutoML
4. Verifying results
5. Conclusions

# 1. Setup

In [7]:
import os
import h2o
import sys
import json
import pickle
import pandas as pd
import numpy as np

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,21 mins 35 secs
H2O_cluster_timezone:,Europe/Paris
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.34.0.3
H2O_cluster_version_age:,19 days
H2O_cluster_name:,H2O_from_python_ff268477_ugo4gy
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.456 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"locked, healthy"


In [9]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.feature_engineering.FeatureEngineering import FeatureEngineering
from src.DataLoader import DataLoader
from src.modeling.Modeling import Modeling
from src.processing.Processing import Processing
from src.utils import read_files

Variables

In [10]:
pd.options.display.max_rows = 6
with open('../config/variables.json') as json_file:
    var = json.load(json_file)
STRUCTURED_DATA = var.get('STRUCTURED_DATA')
RAW_DATA = var.get('RAW_DATA')

Objects

In [11]:
feng = FeatureEngineering()
modeling = Modeling()
processing = Processing()
load = DataLoader(RAW_DATA)

# 2. Loading Data

In [20]:
fits = load.load_fits(read_files(RAW_DATA))
energies = processing.get_energy_dict(fits)
pca_object = pickle.load(open(STRUCTURED_DATA+"pca.pkl","rb"))
test_data = pd.read_pickle(STRUCTURED_DATA + "dataframe_1200-combinations_33890-photons.pkl")

In [77]:
models = {
    label: h2o.import_mojo(STRUCTURED_DATA + f"{label}_best-model_3-models_seed-1")
for label in energies.keys()}

generic Model Build progress: |██████████████████████████████████████████████████| (done) 100%
Model Details
H2OGenericEstimator :  Import MOJO Model
Model Key:  Generic_model_python_1635253645657_21218


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,435.0,435.0,497990.0,1.0,15.0,13.018391,2.0,149.0,86.308044




ModelMetricsBinomialGeneric: generic
** Reported on train data. **

MSE: 0.0
RMSE: 0.0
LogLoss: 0.0
Mean Per-Class Error: 0.0
AUC: 1.0
AUCPR: 1.0
Gini: 1.0

Confusion Matrix (Act/Pred) for max f1 @ threshold = 1.0: 


,,False,True,Error,Rate
0,False,18000.0,0.0,0.0,(0.0/18000.0)
1,True,0.0,7200.0,0.0,(0.0/7200.0)
2,Total,18000.0,7200.0,0.0,(0.0/25200.0)



Maximum Metrics: Maximum metrics at their respective thresholds


...


See the whole table with table.as_data_frame()

Scoring History: 


,,timestamp,duration,iteration,lambda,predictors,deviance_train,deviance_xval,deviance_se,alpha,iterations,training_rmse,training_logloss,training_r2,training_auc,training_pr_auc,training_lift,training_classification_error
0,,2021-10-26 15:23:08,0.000 sec,2,.31E2,8,1.183569,1.186157,0.000162,0.0,None,,,,,,,
1,,2021-10-26 15:23:08,0.020 sec,4,.19E2,9,1.175778,1.179901,0.000162,0.0,None,,,,,,,
2,,2021-10-26 15:23:08,0.040 sec,6,.12E2,11,1.163426,1.169955,0.000161,0.0,None,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17,,2021-10-26 15:23:08,0.333 sec,42,.95E-2,40,0.054737,0.065251,0.000024,0.0,None,,,,,,,
18,,2021-10-26 15:23:08,0.355 sec,45,.59E-2,40,0.037532,0.044906,0.000018,0.0,None,,,,,,,
19,,2021-10-26 15:23:08,0.376 sec,48,.37E-2,40,0.025556,0.030675,0.000013,0.0,None,,,,,,,



See the whole table with table.as_data_frame()

Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,2,4.942804,1.000000,0.373238
1,1,4.209114,0.851564,0.317836
2,3,2.957772,0.598400,0.223345
...,...,...,...,...
17,11,0.001614,0.000326,0.000122
18,30,0.001313,0.000266,0.000099
19,25,0.001188,0.000240,0.000090



See the whole table with table.as_data_frame()



# 3. Evaluating Confusion Matrices

In [85]:
for element, model in models.items():
    print(element, "\n", model.confusion_matrix())

eu 
 
Confusion Matrix (Act/Pred) for max f1 @ threshold = 1.0: 


,,False,True,Error,Rate
0,False,18000.0,0.0,0.0,(0.0/18000.0)
1,True,0.0,7200.0,0.0,(0.0/7200.0)
2,Total,18000.0,7200.0,0.0,(0.0/25200.0)



bd 
 
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.9990259008864969: 


,,False,True,Error,Rate
0,False,18000.0,0.0,0.0,(0.0/18000.0)
1,True,0.0,7200.0,0.0,(0.0/7200.0)
2,Total,18000.0,7200.0,0.0,(0.0/25200.0)



cs 
 
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.99549422806374: 


...

,,False,True,Error,Rate
0,False,18000.0,0.0,0.0,(0.0/18000.0)
1,True,0.0,7200.0,0.0,(0.0/7200.0)
2,Total,18000.0,7200.0,0.0,(0.0/25200.0)



ba 
 
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.9993511318691692: 


,,False,True,Error,Rate
0,False,18000.0,0.0,0.0,(0.0/18000.0)
1,True,0.0,7200.0,0.0,(0.0/7200.0)
2,Total,18000.0,7200.0,0.0,(0.0/25200.0)



co 
 
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.9992406651234982: 


,,False,True,Error,Rate
0,False,18000.0,0.0,0.0,(0.0/18000.0)
1,True,0.0,7200.0,0.0,(0.0/7200.0)
2,Total,18000.0,7200.0,0.0,(0.0/25200.0)


Probably there is something wrong, also it's strange because the total is varying sometimes, and it was trained with a 25200 rows DataFrame

# 4. Testing on new generated DataFrame

In [46]:
df_test = pd.DataFrame(0.0, index=test_data.index, columns = [str(float(col)) for col in range(1000)])
df_test.update(test_data)

In [47]:
df_test

,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,...,990.0,991.0,992.0,993.0,994.0,995.0,996.0,997.0,998.0,999.0
eu_bd_0,0.0,0.000059,0.000074,0.000782,0.004382,0.009649,0.009693,0.008734,0.007362,0.009796,...,0.000015,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
eu_bd_6,0.0,0.000074,0.000162,0.000885,0.004456,0.009472,0.010180,0.008941,0.007628,0.009678,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
eu_bd_13,0.0,0.000074,0.000059,0.000708,0.004411,0.009413,0.010977,0.008660,0.007628,0.010047,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ba_co_1197,0.0,0.000000,0.000000,0.000000,0.000015,0.000030,0.000162,0.002847,0.013145,0.016598,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ba_co_1198,0.0,0.000000,0.000000,0.000015,0.000015,0.000015,0.000162,0.002479,0.013721,0.016258,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ba_co_1199,0.0,0.000000,0.000000,0.000015,0.000059,0.000000,0.000148,0.002346,0.013057,0.015934,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 4.1 Applying same PCA

In [48]:
df_features_test = pd.DataFrame(processing.apply_pca(df_test, pca_object), index=df_test.index)

### 4.1.1 transforming into h2o object

In [68]:
df_h2o_test = h2o.H2OFrame(df_features_test)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [69]:
df_h2o_test

0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,has_eu,has_bd,has_cs,has_sp,has_am,has_ba,has_co
-0.00344062,0.0727636,0.0763169,0.0161553,-0.0106857,0.022407,-0.000797371,0.000163216,9.57119e-06,-5.18038e-05,0.000798879,0.0004633,-0.000575878,-0.00112155,-0.000268586,0.000235237,0.00015935,0.000291408,-0.000102473,-0.000184702,-0.000202093,-0.000296824,0.000362572,4.61721e-05,-0.000487071,-0.000384254,-1.83246e-05,0.000330707,-0.000221953,0.000381767,0.000335805,-7.41577e-05,0.000101075,-0.000231792,-0.000216238,-1.04581e-05,2.86744e-05,-0.000343882,0.000411245,-0.000196385,-0.000327261,-0.000325088,True,True,False,False,False,False,False
-0.00277989,0.0726063,0.0740955,0.0163213,-0.0101863,0.0223433,0.000236895,2.48485e-06,-0.000343811,0.00013181,-0.000728263,-0.000168958,-0.000193713,0.000132782,-0.000150278,0.000117792,-0.000115176,-0.000122532,-0.000316776,6.82314e-05,1.74786e-05,-4.08924e-05,0.000672659,-0.000233589,0.00028235,-0.000462656,-0.000424004,0.000162591,-0.000105221,-0.000149985,0.000134152,-0.000516625,0.000362012,0.000355928,0.000253784,-0.000384108,0.000112895,0.000215591,-7.25507e-05,-0.000193842,0.00040201,-0.000356739,True,True,False,False,False,False,False
-0.00327916,0.0726058,0.0770387,0.0165674,-0.0111866,0.0222695,-0.000988526,0.000168905,0.000255901,-0.000160286,0.000168304,0.000272889,0.000122109,-0.000283661,-0.000799597,0.000251314,0.000129732,0.000249672,0.000169563,0.000748985,2.18639e-06,0.0006146,7.37384e-05,-0.000427976,-0.000465643,-0.000258255,-0.000740104,0.000106953,-0.000118501,0.000477697,0.000193758,0.000348413,-0.000360315,-7.21476e-05,-0.000341342,-0.000368723,-0.000300774,0.000211261,-3.21165e-05,0.000425577,-0.000180334,-0.000322085,True,True,False,False,False,False,False
-0.00273888,0.0718412,0.0749954,0.0172429,-0.011026,0.0217354,0.000478823,0.00024454,0.000233026,0.000131378,0.000807876,-0.000366217,-8.54864e-05,0.00109387,0.000693139,-0.000435137,5.75283e-05,1.13644e-05,0.000447466,0.00012385,-0.0012131,-0.000150198,0.000456306,0.000586746,-7.85917e-05,-0.000932848,-0.000110999,8.58056e-06,3.08378e-05,0.00013624,0.000386285,-0.000151142,0.000339391,0.000140195,4.24366e-05,0.000490645,0.00036321,6.9532e-06,0.000611603,-0.000607195,0.000559689,1.93641e-05,True,True,False,False,False,False,False
-0.00334643,0.0724632,0.0757217,0.016561,-0.0112497,0.0220739,-0.000162583,0.000483694,0.00020916,-6.8631e-05,0.000987558,-0.000358487,-6.90815e-05,0.000383845,0.000605022,5.29925e-05,2.92639e-05,0.000108455,-0.00074527,0.00029743,-0.00039726,0.000795817,-0.000114107,1.06534e-05,0.000386196,-3.95739e-05,6.01492e-06,0.000100441,-0.000234277,-0.000280429,0.00018505,0.000377691,-0.000242175,7.52625e-05,-0.000144622,2.2729e-05,0.000222773,-6.12541e-05,3.81242e-05,3.12162e-05,-5.12754e-05,9.89287e-05,True,True,False,False,False,False,False
-0.00298655,0.0727042,0.0747158,0.0166942,-0.0108299,0.0223383,0.000286406,-0.000213667,-0.000213921,0.000236888,-0.00111214,0.000270712,0.000571057,-0.000142363,0.000922589,0.000290238,-0.000278761,9.77176e-05,-9.03485e-05,-9.87387e-05,0.00050846,0.0002955,-0.000308713,0.000459487,-0.000107278,0.000173787,0.000317417,-0.000450767,6.64463e-06,0.000268252,-8.67172e-05,0.000309182,0.00015692,-6.26975e-05,0.000276419,-0.000113783,0.000533856,-0.000539553,-0.00021249,0.000354429,9.12743e-05,-0.000115382,True,True,False,False,False,False,False
-0.00314245,0.0719557,0.0747799,0.0168176,-0.0103692,0.0223549,0.00069074,-0.000430116,0.000168154,0.000467032,-0.00121026,0.00025713,0.000201816,-7.70523e-05,-0.000148767,-0.000195973,-0.000399498,8.10279e-05,4.96887e-06,-0.000266239,5.20801e-05,4.85486e-05,-0.000218168,0.000611347,-0.000304609,0.000416249,-0.000154777,-3.8713e-05,0.000325295,4.04756e-05,-0.000268815,0.000408896,1.8545e-05,-1.00919e-05,-5.22618e-05,0.000230265,0.000232545,-0.000117983,-0.000177011,8.35199e-05,-0.00053008,-9.39737e-05,True,True,False,False,False,False,False
-0.00264531,0.

## 4.2 Labeling DataFrame

In [54]:
for label in energies.keys():
    feng.label_dataframe(df_features_test, label)

In [55]:
df_features_test

,0,1,2,3,4,5,6,7,8,9,...,39,40,41,has_eu,has_bd,has_cs,has_sp,has_am,has_ba,has_co
eu_bd_0,-0.003441,0.072764,0.076317,0.016155,-0.010686,0.022407,-0.000797,0.000163,0.000010,-0.000052,...,-0.000196,-0.000327,-0.000325,True,True,False,False,False,False,False
eu_bd_6,-0.002780,0.072606,0.074096,0.016321,-0.010186,0.022343,0.000237,0.000002,-0.000344,0.000132,...,-0.000194,0.000402,-0.000357,True,True,False,False,False,False,False
eu_bd_13,-0.003279,0.072606,0.077039,0.016567,-0.011187,0.022270,-0.000989,0.000169,0.000256,-0.000160,...,0.000426,-0.000180,-0.000322,True,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ba_co_1197,-0.075956,0.002689,-0.072538,-0.060299,-0.021543,-0.001522,0.000393,0.000692,-0.000559,0.000625,...,-0.000432,-0.000060,-0.000586,False,False,False,False,False,True,True
ba_co_1198,-0.076040,0.003270,-0.072998,-0.060762,-0.020968,-0.001899,0.000656,0.001036,-0.000509,0.001102,...,-0.000234,0.000064,0.000629,False,False,False,False,False,True,True
ba_co_1199,-0.075506,0.003745,-0.072726,-0.059750,-0.020189,-0.002157,0.001429,0.001504,-0.000962,0.001078,...,0.000022,-0.000011,0.000330,False,False,False,False,False,True,True


## 4.3 Predicting

In [78]:
{
    
}res = models["am"].predict(df_h2o_test).as_data_frame()

generic prediction progress: |███████████████████████████████████████████████████| (done) 100%


In [87]:
res.predict.sum()

6766

In [73]:
df_h2o_test

0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,has_eu,has_bd,has_cs,has_sp,has_am,has_ba,has_co
-0.00344062,0.0727636,0.0763169,0.0161553,-0.0106857,0.022407,-0.000797371,0.000163216,9.57119e-06,-5.18038e-05,0.000798879,0.0004633,-0.000575878,-0.00112155,-0.000268586,0.000235237,0.00015935,0.000291408,-0.000102473,-0.000184702,-0.000202093,-0.000296824,0.000362572,4.61721e-05,-0.000487071,-0.000384254,-1.83246e-05,0.000330707,-0.000221953,0.000381767,0.000335805,-7.41577e-05,0.000101075,-0.000231792,-0.000216238,-1.04581e-05,2.86744e-05,-0.000343882,0.000411245,-0.000196385,-0.000327261,-0.000325088,True,True,False,False,False,False,False
-0.00277989,0.0726063,0.0740955,0.0163213,-0.0101863,0.0223433,0.000236895,2.48485e-06,-0.000343811,0.00013181,-0.000728263,-0.000168958,-0.000193713,0.000132782,-0.000150278,0.000117792,-0.000115176,-0.000122532,-0.000316776,6.82314e-05,1.74786e-05,-4.08924e-05,0.000672659,-0.000233589,0.00028235,-0.000462656,-0.000424004,0.000162591,-0.000105221,-0.000149985,0.000134152,-0.000516625,0.000362012,0.000355928,0.000253784,-0.000384108,0.000112895,0.000215591,-7.25507e-05,-0.000193842,0.00040201,-0.000356739,True,True,False,False,False,False,False
-0.00327916,0.0726058,0.0770387,0.0165674,-0.0111866,0.0222695,-0.000988526,0.000168905,0.000255901,-0.000160286,0.000168304,0.000272889,0.000122109,-0.000283661,-0.000799597,0.000251314,0.000129732,0.000249672,0.000169563,0.000748985,2.18639e-06,0.0006146,7.37384e-05,-0.000427976,-0.000465643,-0.000258255,-0.000740104,0.000106953,-0.000118501,0.000477697,0.000193758,0.000348413,-0.000360315,-7.21476e-05,-0.000341342,-0.000368723,-0.000300774,0.000211261,-3.21165e-05,0.000425577,-0.000180334,-0.000322085,True,True,False,False,False,False,False
-0.00273888,0.0718412,0.0749954,0.0172429,-0.011026,0.0217354,0.000478823,0.00024454,0.000233026,0.000131378,0.000807876,-0.000366217,-8.54864e-05,0.00109387,0.000693139,-0.000435137,5.75283e-05,1.13644e-05,0.000447466,0.00012385,-0.0012131,-0.000150198,0.000456306,0.000586746,-7.85917e-05,-0.000932848,-0.000110999,8.58056e-06,3.08378e-05,0.00013624,0.000386285,-0.000151142,0.000339391,0.000140195,4.24366e-05,0.000490645,0.00036321,6.9532e-06,0.000611603,-0.000607195,0.000559689,1.93641e-05,True,True,False,False,False,False,False
-0.00334643,0.0724632,0.0757217,0.016561,-0.0112497,0.0220739,-0.000162583,0.000483694,0.00020916,-6.8631e-05,0.000987558,-0.000358487,-6.90815e-05,0.000383845,0.000605022,5.29925e-05,2.92639e-05,0.000108455,-0.00074527,0.00029743,-0.00039726,0.000795817,-0.000114107,1.06534e-05,0.000386196,-3.95739e-05,6.01492e-06,0.000100441,-0.000234277,-0.000280429,0.00018505,0.000377691,-0.000242175,7.52625e-05,-0.000144622,2.2729e-05,0.000222773,-6.12541e-05,3.81242e-05,3.12162e-05,-5.12754e-05,9.89287e-05,True,True,False,False,False,False,False
-0.00298655,0.0727042,0.0747158,0.0166942,-0.0108299,0.0223383,0.000286406,-0.000213667,-0.000213921,0.000236888,-0.00111214,0.000270712,0.000571057,-0.000142363,0.000922589,0.000290238,-0.000278761,9.77176e-05,-9.03485e-05,-9.87387e-05,0.00050846,0.0002955,-0.000308713,0.000459487,-0.000107278,0.000173787,0.000317417,-0.000450767,6.64463e-06,0.000268252,-8.67172e-05,0.000309182,0.00015692,-6.26975e-05,0.000276419,-0.000113783,0.000533856,-0.000539553,-0.00021249,0.000354429,9.12743e-05,-0.000115382,True,True,False,False,False,False,False
-0.00314245,0.0719557,0.0747799,0.0168176,-0.0103692,0.0223549,0.00069074,-0.000430116,0.000168154,0.000467032,-0.00121026,0.00025713,0.000201816,-7.70523e-05,-0.000148767,-0.000195973,-0.000399498,8.10279e-05,4.96887e-06,-0.000266239,5.20801e-05,4.85486e-05,-0.000218168,0.000611347,-0.000304609,0.000416249,-0.000154777,-3.8713e-05,0.000325295,4.04756e-05,-0.000268815,0.000408896,1.8545e-05,-1.00919e-05,-5.22618e-05,0.000230265,0.000232545,-0.000117983,-0.000177011,8.35199e-05,-0.00053008,-9.39737e-05,True,True,False,False,False,False,False
-0.00264531,0.

In [44]:
X = h2o.H2OFrame(df_features_test)# df.loc[:, df.columns != target_variable]
y = df_features_test.loc[:, "has_sp"]

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [ ]:
model.predict(test_boston.head(3))